In [1]:
import pandas as pd

import sys
sys.path.append('../')
from utils import *

pd.set_option('display.max_columns', None)

In [2]:
DATA_PATH = '../../data/'
WYSCOUT_PATH = DATA_PATH + 'wyscout/'
SKILLCORNER_PATH = DATA_PATH + 'skillcorner/'
SKILLCORNER_RAW_PATH = DATA_PATH + 'skillcorner_raw/'
XT_PLOT_PATH = DATA_PATH + 'xt_plot_worldcup.csv'
MATCH_IDS_PATH = DATA_PATH + 'matchids.csv'

## Load Data

In [3]:
match_info = pd.read_csv(MATCH_IDS_PATH)
match_info = match_info.dropna(subset=['wyscout', 'skillcorner'])
match_info['skillcorner'] = match_info['skillcorner'].astype(int)

In [4]:
match_info.head(5)

,wyscout,skillcorner,date,home,result,away
8,5414111,952209,2023-03-24,France,4-0,Netherlands
10,5414113,726849,2023-03-24,Czech Republic,3-1,Poland
12,5414115,722109,2023-03-24,Sweden,0-3,Belgium
25,5414127,726848,2023-03-26,England,2-0,Ukraine
33,5414135,952208,2023-03-27,Republic of Ireland,0-1,France


In [5]:
wyscout_id = match_info['wyscout'].values[0]
skillcorner_id = match_info['skillcorner'].values[0]

### wyscout

In [6]:
wyscout_data = wyscout_to_df(WYSCOUT_PATH + str(wyscout_id) + ".json")
with pd.option_context('display.max_columns', None):
    display(wyscout_data.head(3))

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type.primary,type.secondary,location.x,location.y,team.id,team.name,team.formation,opponentTeam.id,opponentTeam.name,opponentTeam.formation,player.id,player.name,player.position,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,possession.id,possession.duration,possession.types,possession.eventsNumber,possession.eventIndex,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,possession.team.id,possession.team.name,possession.team.formation,possession.attack,pass,carry.progression,carry.endLocation.x,carry.endLocation.y,groundDuel.opponent.id,groundDuel.opponent.name,groundDuel.opponent.position,groundDuel.duelType,groundDuel.keptPossession,groundDuel.progressedWithBall,groundDuel.stoppedProgress,groundDuel.recoveredPossession,groundDuel.takeOn,groundDuel.side,groundDuel.relatedDuelId,possession.attack.withShot,possession.attack.withShotOnGoal,possession.attack.withGoal,possession.attack.flank,possession.attack.xg,shot.bodyPart,shot.isGoal,shot.onTarget,shot.goalZone,shot.xg,shot.postShotXg,shot.goalkeeperActionId,shot.goalkeeper.id,shot.goalkeeper.name,possession,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.yellowCard,infraction.redCard,infraction.type,infraction.opponent.id,infraction.opponent.name,infraction.opponent.position,infraction.opponent
0,1587492614,5414111,1H,0,1,00:00:01.912,3.912638,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",51,50,664,Netherlands,4-2-3-1,4418,France,4-3-3,118,M. Depay,CF,True,114.0,None,24.0,625.0,M. de Roon,RDMF,42.0,83.0,1.587493e+09,110.275779,[],43.0,0.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1587492615,5414111,1H,0,5,00:00:05.782,7.782226,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",42,83,664,Netherlands,4-2-3-1,4418,France,4-3-3,625,M. de Roon,RDMF,True,-133.0,None,21.0,476361.0,L. Geertruida,RCB,29.0,60.0,1.587493e+09,110.275779,[],43.0,1.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1587492616,5414111,1H,0,8,00:00:08.951,10.951834,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",29,60,664,Netherlands,4-2-3-1,4418,France,4-3-3,476361,L. Geertruida,RCB,True,-100.0,None,22.0,370.0,V. van Dijk,LCB,25.0,28.0,1.587493e+09,110.275779,[],43.0,2.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### skillcorner

In [7]:
metadata = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_metadata.csv")
tracking_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_tracking.csv")
lineup_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_lineup.csv")

In [8]:
print("Metadata")
display(metadata.head(3))
print("Tracking")
display(tracking_df.head(3))
print("Lineup")
display(lineup_df.head(3))

Metadata


,match_id,match_date,competition,season,home_team,away_team,home_score,away_score,home_team_jersey_color,away_team_jersey_color,home_team_number_color,away_team_number_color,home_team_coach,away_team_coach,pitch_name,pitch_length,pitch_width,provider,fps
0,952209,03/24/2023 19:45,European Championship Qualifiers,2023/2024,France,Netherland,4,0,#fafafa,#ff824c,#000000,#000000,Didier Deschamps,Frank De Boer,Stade de France,105.0,68.0,SkillCorner,10


Tracking


,match_id,half,frame_id,timestamp,object_id,x,y,z,extrapolated
0,952209,1,20,0,-1,0.12,-0.19,0.12,False
1,952209,1,20,0,7604,-39.44,-0.21,0.00,True
2,952209,1,20,0,16320,-16.05,-7.18,0.00,False


Lineup


,match_id,team_name,player_id,player_first_name,player_last_name,player_shirt_number,player_position,player_birthdate,start_time,end_time,yellow_card,red_card,injured,goal,own_goal
0,952209,Netherland,14397,Tyrell,Malacia,16,Right Wing Back,1999-08-17,01:26:59,NaN,0,0,False,0,0
1,952209,Netherland,12654,Donyell,Malen,7,Right Winger,1999-01-19,01:06:22,NaN,0,0,False,0,0
2,952209,Netherland,2548,Davy,Klaassen,14,Attacking Midfield,1993-02-21,01:06:22,NaN,0,0,False,0,0


# Pass Events

In [9]:
passes_df = wyscout_data[wyscout_data['type.primary'] == 'pass']

# Filter out inaccurate passes
accurate_only = True
if accurate_only:
    passes_df = passes_df[passes_df['pass.accurate'] == True]

print(passes_df.shape)

(897, 93)


In [10]:
passes = passes_df.loc[:,['matchId', 'matchTimestamp', 'matchPeriod', 'team.id', 'team.name', 'player.id', 'player.name', 'opponentTeam.id', 'opponentTeam.name', 'pass.recipient.id', 'pass.recipient.name', 'location.x', 'location.y', 'pass.endLocation.x', 'pass.endLocation.y']]
passes.head()

,matchId,matchTimestamp,matchPeriod,team.id,team.name,player.id,player.name,opponentTeam.id,opponentTeam.name,pass.recipient.id,pass.recipient.name,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,5414111,00:00:01.912,1H,664,Netherlands,118,M. Depay,4418,France,625.0,M. de Roon,51,50,42.0,83.0
1,5414111,00:00:05.782,1H,664,Netherlands,625,M. de Roon,4418,France,476361.0,L. Geertruida,42,83,29.0,60.0
2,5414111,00:00:08.951,1H,664,Netherlands,476361,L. Geertruida,4418,France,370.0,V. van Dijk,29,60,25.0,28.0
3,5414111,00:00:13.151,1H,664,Netherlands,370,V. van Dijk,4418,France,476361.0,L. Geertruida,25,28,25.0,51.0
5,5414111,00:00:17.930,1H,664,Netherlands,476361,L. Geertruida,4418,France,370.0,V. van Dijk,32,50,30.0,17.0


In [11]:
passes['matchPeriod'] = passes['matchPeriod'].apply(lambda x: int(x.split('H')[0]))
passes['pass.recipient.id'] = passes['pass.recipient.id'].astype(int)
passes['pass.endLocation.x'] = passes['pass.endLocation.x'].astype(int)
passes['pass.endLocation.y'] = passes['pass.endLocation.y'].astype(int)

In [12]:
passes.head()

,matchId,matchTimestamp,matchPeriod,team.id,team.name,player.id,player.name,opponentTeam.id,opponentTeam.name,pass.recipient.id,pass.recipient.name,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,5414111,00:00:01.912,1,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,51,50,42,83
1,5414111,00:00:05.782,1,664,Netherlands,625,M. de Roon,4418,France,476361,L. Geertruida,42,83,29,60
2,5414111,00:00:08.951,1,664,Netherlands,476361,L. Geertruida,4418,France,370,V. van Dijk,29,60,25,28
3,5414111,00:00:13.151,1,664,Netherlands,370,V. van Dijk,4418,France,476361,L. Geertruida,25,28,25,51
5,5414111,00:00:17.930,1,664,Netherlands,476361,L. Geertruida,4418,France,370,V. van Dijk,32,50,30,17


# Compute ΔxT

In [13]:
xt_table = pd.read_csv(DATA_PATH + 'smoothed_xt.csv')

In wyscout coordinates are normalized to the range (0,100) in both spatial dimensions.

In [14]:
cell_width = 100 / xt_table.shape[1]
cell_height = 100 / xt_table.shape[0]

print(f'cell width:\t{round(cell_width,2)}')
print(f'cell height:\t{round(cell_height,2)}')

cell width:	0.95
cell height:	1.47


In [15]:
def get_xt_index(x, y):
    x_index = min(int(x // cell_width), xt_table.shape[1] - 1)
    y_index = min(int(y // cell_height), xt_table.shape[0] - 1)
    return x_index, y_index

In [16]:
start_xts = passes.apply(lambda row: xt_table.iat[get_xt_index(row['location.x'], row['location.y'])[1], 
                                                           get_xt_index(row['location.x'], row['location.y'])[0]], axis=1)
end_xts = passes.apply(lambda row: xt_table.iat[get_xt_index(row['pass.endLocation.x'], row['pass.endLocation.y'])[1], 
                                                         get_xt_index(row['pass.endLocation.x'], row['pass.endLocation.y'])[0]], axis=1)
passes.loc[:,'dxt'] = end_xts - start_xts

# Player Locations

In [17]:
columns_to_drop = ['frame_id', 'extrapolated']
existing_columns_to_drop = [col for col in columns_to_drop if col in tracking_df.columns]
tracking_df.drop(columns=existing_columns_to_drop, inplace=True)

In [18]:
if not {'timestamp', 'period'}.issubset(tracking_df.index.names):    
    tracking_df['timestamp'] = tracking_df.apply(standardize_timestamp, axis=1)
    tracking_df.set_index(['timestamp', 'half'], inplace=True)
    tracking_df.rename_axis(index={'timestamp': 'timestamp', 'half': 'period'}, inplace=True)
tracking_df.sample(3)

,,match_id,object_id,x,y,z
timestamp,period,,,,,
00:35:04.500000,1,952209,11606,26.23,3.19,0.0
00:28:45.700000,1,952209,31930,32.90,-28.84,0.0
01:31:02.400000,2,952209,12654,12.34,-20.81,0.0


In [19]:
passes['quantizedTimestamp'] = passes['matchTimestamp'].apply(round_to_tenth_of_second)
if not {'timestamp', 'period'}.issubset(passes.index.names):
    passes.set_index(['quantizedTimestamp', 'matchPeriod'], inplace=True)
    passes.rename_axis(index={'quantizedTimestamp': 'timestamp', 'matchPeriod': 'period'}, inplace=True)

In [20]:
joined_df = passes.join(tracking_df, how='inner', validate='one_to_many')

In [21]:
columns_to_drop = ['match_id']
existing_columns_to_drop = [col for col in columns_to_drop if col in joined_df.columns]
joined_df.drop(columns=existing_columns_to_drop, inplace=True)

joined_df['object_id'] = joined_df['object_id'].astype(int)

# Normalize Pitch Coordinates

In [22]:
pitch_length = metadata['pitch_length'].values[0]
pitch_width = metadata['pitch_width'].values[0]

In [23]:
start_locations = joined_df.apply(
    lambda row: wyscout_to_pitch(row['location.x'], row['location.y'], pitch_length, pitch_width), 
    axis=1)
end_locations = joined_df.apply(
    lambda row: wyscout_to_pitch(row['pass.endLocation.x'], row['pass.endLocation.y'], pitch_length, pitch_width), 
    axis=1)
joined_df[['location.x', 'location.y']] = start_locations.apply(pd.Series)
joined_df[['pass.endLocation.x', 'pass.endLocation.y']] = end_locations.apply(pd.Series)

# Identify Attackers and Defenders

In [24]:
lineup_df = lineup_df[['team_name', 'player_id', 'player_first_name', 'player_last_name']]

In [25]:
joined_df.reset_index(inplace=True)
joined_df.drop(columns=['matchTimestamp'], inplace=True)

In [26]:
joined_df = joined_df.merge(lineup_df, left_on='object_id', right_on='player_id')

In [27]:
joined_df

,timestamp,period,matchId,team.id,team.name,player.id,player.name,opponentTeam.id,opponentTeam.name,pass.recipient.id,pass.recipient.name,location.x,location.y,pass.endLocation.x,pass.endLocation.y,dxt,object_id,x,y,z,team_name,player_id,player_first_name,player_last_name
0,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,7604,-41.03,-0.05,0.0,France,7604,Mike,Maignan
1,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,16320,-15.86,-7.61,0.0,France,16320,Ibrahima,Konaté
2,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,11516,-16.08,2.42,0.0,France,11516,Dayot,Upamecano
3,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,11630,-14.54,-18.87,0.0,France,11630,Jules,Koundé
4,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,11606,-14.08,17.56,0.0,France,11606,Theo,Hernández
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19707,01:33:38.800000,2,5414111,664,Netherlands,118,M. Depay,4418,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,20191,31.49,-1.03,0.0,France,20191,Khéphren,Thuram-Ulien
19708,01:33:38.800000,2,5414111,664,Netherlands,118,M. Depay,4418,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,12229,31.89,8.13,0.0,France,12229,Youssouf,Fofana
19709,01:33:38.800000,2,5414111,664,Netherlands,118,M. Depay,4418,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,12246,35.56,12.33,0.0,France,12246,Moussa,Diaby
19710,01:33:38.800000,2,5414111,664,Netherlands,118,M. Depay,4418,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,6028,5.09,-7.70,0.0,France,6028,Kylian,Mbappé Lottin


In [28]:
columns_to_prefix = ['object_id', 'x', 'y', 'z', 'x_norm', 'y_norm', 'team_name', 'player_id' ,'player_first_name', 'player_last_name']
prefix = 'tracking.'
joined_df.rename(columns={col: prefix + col for col in columns_to_prefix}, inplace=True)

In [29]:
joined_df

,timestamp,period,matchId,team.id,team.name,player.id,player.name,opponentTeam.id,opponentTeam.name,pass.recipient.id,pass.recipient.name,location.x,location.y,pass.endLocation.x,pass.endLocation.y,dxt,tracking.object_id,tracking.x,tracking.y,tracking.z,tracking.team_name,tracking.player_id,tracking.player_first_name,tracking.player_last_name
0,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,7604,-41.03,-0.05,0.0,France,7604,Mike,Maignan
1,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,16320,-15.86,-7.61,0.0,France,16320,Ibrahima,Konaté
2,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,11516,-16.08,2.42,0.0,France,11516,Dayot,Upamecano
3,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,11630,-14.54,-18.87,0.0,France,11630,Jules,Koundé
4,00:00:01.900000,1,5414111,664,Netherlands,118,M. Depay,4418,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,11606,-14.08,17.56,0.0,France,11606,Theo,Hernández
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19707,01:33:38.800000,2,5414111,664,Netherlands,118,M. Depay,4418,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,20191,31.49,-1.03,0.0,France,20191,Khéphren,Thuram-Ulien
19708,01:33:38.800000,2,5414111,664,Netherlands,118,M. Depay,4418,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,12229,31.89,8.13,0.0,France,12229,Youssouf,Fofana
19709,01:33:38.800000,2,5414111,664,Netherlands,118,M. Depay,4418,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,12246,35.56,12.33,0.0,France,12246,Moussa,Diaby
19710,01:33:38.800000,2,5414111,664,Netherlands,118,M. Depay,4418,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,6028,5.09,-7.70,0.0,France,6028,Kylian,Mbappé Lottin
